# Lunchanator: The lunch deciding utility

#### Recommendation for a lunch group

Importing needed packages

In [3]:
import requests
import json
import geocoder
import numpy as np
import pandas as pd

We used the Bing geocoder because the Google limit for the month was reached via other projects.

In [14]:
g = geocoder.bing('Brooklyn, NY', key='Aj6iRGHOxlWE8l0T9VNPg9WCWv9jKQdeEk1P5bd-bqMaHtf8ukyJwAbChFDEfYFJ')

Foursquare request for restaurants in the Brooklyn, NY area.

In [17]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id = '0GPMCSTZ5SOR20WJQNQWRXM0YHXQDMIKAQKHMTPQM2FPHPES', # your Foursquare ID
  client_secret = 'N2VIOQ1Y45VH2MCATYXQJQZS2AEALSIC4UPVWKUGBFU0GSZS', # your Foursquare Secret
  v = '20180605', # Foursquare API version
  ll = str(g.json['lat']) + ',' + str(g.json['lng']),
  query = 'restaurant',
  radius = 1000,
  limit = 50,
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

Parsing results into a list containing restaurant names.

In [18]:
restaurants = []
for m in data['response']['groups'][0]['items']:
    restaurants.append(m['venue']['name'])

This block generates a pretend dataset of 20 users each having selected 15 "good" and 15 "bad" restaurants each. Note that no user can have the same restaurant in both categories.

In [19]:
d = {}
d['users'] = ['user'+str(i) for i in range(20)]
d['good restaurants'] = []
d['bad restaurants'] = []
for i in range(1,21):
    restaurant_list = np.random.choice(restaurants, size=30, replace=False)
    d['good restaurants'].append(restaurant_list[:15])
    d['bad restaurants'].append(restaurant_list[15:])

Transforming the above generated data into a dataframe.

In [20]:
user_df = pd.DataFrame.from_dict(d)

In [21]:
user_df = user_df[['users','good restaurants','bad restaurants']]

An example of user data.

In [22]:
user_df.head()

,users,good restaurants,bad restaurants
0,user0,"[Doner Kebab NYC, Beasts & Bottles, Wild Ginge...","[One Girl Cookies, Pio Bagel, Sushi Gallery, F..."
1,user1,"[Henry Public, two8two Bar & Burger, sweetgree...","[Sottocasa Pizzeria, Chez Moi, Panera Bread, W..."
2,user2,"[Iron Chef House, Oh! Dumplings, Doner Kebab N...","[Grand Army, Shelsky's of Brooklyn, Court Stre..."
3,user3,"[Sushi Gallery, Hibino, Piz-zetta, Lassen & He...","[Court Street Bagels, My Little Pizzeria, Bar ..."
4,user4,"[Sushi Gallery, Grand Army, Lillo, Pio Bagel, ...","[Rocco's Tacos and Tequila Bar Brooklyn, Bareb..."


Creating a group of 6 people at random that wish to have lunch together.

In [23]:
df_lunch_group = user_df.sample(n=6)

In [24]:
df_lunch_group = df_lunch_group.sort_index()

The lunch group.

In [25]:
df_lunch_group

,users,good restaurants,bad restaurants
12,user12,"[Fast and Fresh Burrito Deli, Gregory's Coffee...","[Rocco's Tacos and Tequila Bar Brooklyn, Doner..."
13,user13,"[Grand Army, Lassen & Hennigs, Maison Kayser, ...","[Hibino, French Louie, My Little Pizzeria, Mit..."
14,user14,"[Iron Chef House, Luzzo's BK, Pio Bagel, La Va...","[The Gumbo Bros, Mile End Delicatessen, Dellar..."
16,user16,"[Rocco's Tacos and Tequila Bar Brooklyn, Sushi...","[La Vara, Lassen & Hennigs, Wild Ginger, Mitou..."
18,user18,"[Five Guys, sweetgreen, Damascus Bread & Pastr...","[Sottocasa Pizzeria, Colonie, Grand Army, Mais..."
19,user19,"[My Little Pizzeria, La Vara, Doner Kebab NYC,...","[Colonie, Bien Cuit, Oh! Dumplings, DeKalb Mar..."


Creating the list of restaurants that will be voted on.

In [26]:
lunch_votes = [(restaurant,0) for restaurant in restaurants]

In [27]:
lunch_dict = dict(lunch_votes)

Tallying votes.

In [28]:
for rest in lunch_dict.keys():
    for good_choice in df_lunch_group['good restaurants']:
        if rest in good_choice:
            lunch_dict[rest] += 1
    for bad_choice in df_lunch_group['bad restaurants']:
        if rest in bad_choice:
            lunch_dict[rest] -= 1

Conversion to properly formatted dataframe.

In [29]:
d = {'restaurants': [], 'votes': []}
for k,v in lunch_dict.items():
    d['restaurants'].append(k)
    d['votes'].append(v)

In [30]:
df_lunch_votes = pd.DataFrame.from_dict(d)

Sorting the list of restaurants by vote count.

In [31]:
df_lunch_votes_sorted = df_lunch_votes.sort_values('votes', ascending = False)

The sorted list.

In [32]:
df_lunch_votes_sorted

,restaurants,votes
2,Shelsky's of Brooklyn,3
5,sweetgreen,3
0,Queen,2
11,Gregory's Coffee,2
45,Sushi Gallery,2
1,Damascus Bread & Pastry Shop,2
13,Hadramout Restaurant,2
12,French Louie,2
25,One Girl Cookies,2
17,Grand Army,1


A restaurant is selected at random from the top 8.

In [33]:
df_lunch_votes_sorted['restaurants'][:8].sample()

1    Damascus Bread & Pastry Shop
Name: restaurants, dtype: object

#### Determining user compatibilities for <code>user 0</code>

We create a dictionary that computes a number from -1 to 1 between <code>user0</code> and all other users. The closer the value is to 1, the more compatible the two users' preferences. Values close to -1 means the users differ significantly.

In [34]:
comp = {}
for user in user_df[user_df['users'] != 'user0'].users:
    mySum = 0
    count = 0
    for restaurant in restaurants:
        a = 0
        b = 0
        if restaurant in user_df['good restaurants'][user_df['users']=='user0'].tolist()[0]:
            a = 1
        elif restaurant in user_df['bad restaurants'][user_df['users']=='user0'].tolist()[0]:
            a = -1
        if restaurant in user_df['good restaurants'][user_df['users']==user].tolist()[0]:
            b = 1
        elif restaurant in user_df['bad restaurants'][user_df['users']==user].tolist()[0]:
            b = -1    
        mySum += a*b
        count += abs(a*b)
    if count:
        comp[user] = round(mySum/count,2)
    else:
        comp[user] = 0

We now extract the users with the highest compatibility score with <code>user0</code>.

In [35]:
compMax = max(comp.values())

In [36]:
for k,v in comp.items():
    if comp[k] == compMax:
        print(k)

user17


Now that we know that <code>user2</code> has the most similar preferences to <code>user0</code>, we extract restaurants in <code>user2</code>'s lists of preferences that are not in <code>user0</code>'s preferences to make recommendations.

In [37]:
A = set(user_df['good restaurants'][user_df['users']=='user0'].tolist()[0])
B = set(user_df['good restaurants'][user_df['users']=='user2'].tolist()[0])
B-A

{'Colonie',
 "Dellarocco's",
 'Iron Chef House',
 'Lillo',
 'Oh! Dumplings',
 "Rocco's Tacos and Tequila Bar Brooklyn",
 'Sushi Gallery',
 'Yemen Cafe'}